In [1]:
import cv2
import numpy as np
from sklearn.metrics import pairwise

In [9]:
background= None
accumulated_weight=0.5

roi_top=20
roi_bott0m=300
roi_right=300
roi_left=600

In [3]:
def calc_accum_avg(frame,accum):
    global background
    if background is None:
        background=frame.copy().astype('float')
        return None
    cv2.accumulateWeight(frame,background,accumulated_weight)

In [4]:
def segment(frame,threshold=25): #binary
    
    diff=cv2.absdiff(background.astype('uint8'),frame)
    
    ret,thresholded=cv2.threshold(diff,threshold,255,cv2.THRESH_BINARY) #threshold=min value
    image,countours,hierarchy=cv2.findCountours(thresholded.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX)
    if len(cou=ntours)==0:
        return None
    
    else:
        #ASSUMING LARGEDT EX CONTOUR IN ROI IS HAND
        hand_segment=max(contours,key=contourArea)
        
        return (thresholded,hand_segment)

In [5]:
def count_fingers(thresholede,hand_segment):
    
    conv_hull=cv2.convexHull(hand_segment)
    #top
    top=tuple(conv_hull[conv_hull[:,:,1].argmin()][0])
    bottom=tuple(conv_hull[conv_hull[:,:,1].argmin()][0])
    left=tuple(conv_hull[conv_hull[:,:,0].argmin()][0])
    right=tuple(conv_hull[conv_hull[:,:,0].argmin()][0])
    cX=(left[0]+right[0])//2
    cY=(top[1]+bottom[1])//2
    distance=pairwise.euclidean_distances([cX,cY],Y=[left,right,top,bottom])[0]
    max_distance=distance.max()
    radius=int(0.9*max_distance)
    circumfrence=(2*np.pi*radius)
    circular_roi=np.zeros(thresholded[:2],dtype='uint8')
    cv2.circle(circular_roi,(cX,cY),radius,255,10)
    circular_roi=cv2.bitwise_and(thresholded,thresholded,mask=circular_roi)
    image,contours,hierarchy=cv2.findCountours(circular_roi.copy(),cv2.RETER_EXTERENAL,cv2.CHAIN_APPROX_NONE)
    count=0
    for cnt in contours:
        (x,y,w,h)=cv2.boundingRect(cnt)
        out_of_wrist=(cY+(cY*0.25))>(y+h)
        limit_points=((circumfrence*0.25)>cnt.shape[0])
        if out_of_wrist and limit_points:
            count+=1
    return count

In [8]:
cam=cv2.VideoCapture(0)

num_frames=0


while True:
    
    
    
    ret,frame=cam.read()
    frame_copy=frame.copy()
    roi=frame[roi_top:roi_bottom,roi_right:roi_left]
    gray=cv2.cvtColor(roi,cv2.COLOR+BGR2GRAY)
    gray=cv2.GaussianBlur(gray,(7,7),0)
    
    if num_frame<60:
        calc_accum_acg(gray,accumlated_weight)
        if num_frame<=59:
            cv2.putText(frame_copy,'WAIT. GETTING BG',(200,200),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
            cv2.imshow('finger count',frame_copy)   
    else: 
        
        hand = segment(gray)
        
        if hand is not None:
            thresholded,hand_segment=hand
            
            cv2.drawContours(frame_copy,[hand_segment+(roi_right,roi_top)],-1(255,0,0),5)
            fingers=count_fingers(thresholded,hand_segment)
            
            cv2.putText(frame_copy,str(fingers),(70,50),cv2.FONT_HERSHEY_SIMPLEX,2)
            cv2.imshow('THRESHOLDED',thresholded)
    cv2.rectangle(frame_copy,(roi_left,roi_top),(roi_right,roi_bottom),(0,0,255),5)
    num_frame+=1
    cv2.imshow('finger count',frame_copy)
    k=cv2.waitKey(1)& 0xFF
    if k==27:
        break
cam.release()
cv2.destroyAllWindows()

NameError: name 'roi_bottom' is not defined